In [5]:
import pandas as pd

In [6]:
IDN_df = pd.read_csv("~/Coding/buntain/politician_twitterid_data/twitterid_IDN.csv", dtype={"twitter": str})
MLS_df = pd.read_csv("~/Coding/buntain/politician_twitterid_data/twitterid_MLS.csv", dtype={"twitter": str})
PHL_df = pd.read_csv("~/Coding/buntain/politician_twitterid_data/twitterid_PHL.csv", dtype={"twitter": str})

politician_twitter_twitterids_IDN = IDN_df[["can_name_clean", "twitter"]].dropna()
politician_twitter_twitterids_MLS = MLS_df[["can_name_clean", "twitter"]].dropna()
politician_twitter_twitterids_PHL = PHL_df[["can_name_clean", "twitter"]].dropna()
politician_twitter_twitterids_IDN.drop_duplicates(inplace=True)
politician_twitter_twitterids_MLS.drop_duplicates(inplace=True)
politician_twitter_twitterids_PHL.drop_duplicates(inplace=True)


most_followed_data_files = [
    '~/Coding/buntain/most_followed_data/TMRC14_APAC_1.csv', '~/Coding/buntain/most_followed_data/TMRC14_APAC_2.csv', 
    '~/Coding/buntain/most_followed_data/TMRC15_APAC_3.csv']
retweet_data_files = [
    '~/Coding/buntain/retweet_data/TMRC14_APAC_1.csv', '~/Coding/buntain/retweet_data/TMRC14_APAC_2.csv', 
    '~/Coding/buntain/retweet_data/TMRC15_APAC_3.csv', '~/Coding/buntain/retweet_data/twitterrei_china_052020.csv',
    '~/Coding/buntain/retweet_data/twitterrei_china_082019_1.csv', '~/Coding/buntain/retweet_data/twitterrei_china_082019_2.csv', 
    '~/Coding/buntain/retweet_data/twitterrei_china_082019_3.csv', '~/Coding/buntain/retweet_data/twitterrei_thailand_092020.csv']
mentioned_data_files = [
    '~/Coding/buntain/mentioned_data/TMRC14_APAC_1.csv', '~/Coding/buntain/mentioned_data/TMRC14_APAC_2.csv', 
    '~/Coding/buntain/mentioned_data/TMRC15_APAC_3.csv', '~/Coding/buntain/mentioned_data/twitterrei_china_052020.csv',
    '~/Coding/buntain/mentioned_data/twitterrei_china_082019_1.csv', '~/Coding/buntain/mentioned_data/twitterrei_china_082019_2.csv', 
    '~/Coding/buntain/mentioned_data/twitterrei_china_082019_3.csv', '~/Coding/buntain/mentioned_data/twitterrei_thailand_092020.csv']


In [7]:
# use dict to keep track of which file the df is from
most_followed_data_dicts = dict()
for followed_data in most_followed_data_files:
    df = pd.read_csv(followed_data, dtype={"userid": str})
    df["userid"] = df["userid"].apply(str.strip)
    cur_dict = df.set_index("userid")["count"].to_dict()
    file_name = followed_data.split("/")
    most_followed_data_dicts[file_name[-1]] = cur_dict
retweet_data_dicts = dict()
for retweet_data in retweet_data_files:
    df = pd.read_csv(retweet_data, dtype={"userid": str})
    df["userid"] = df["userid"].apply(str.strip)
    cur_dict = df.set_index("userid")["count"].to_dict()
    file_name = followed_data.split("/")
    retweet_data_dicts[file_name[-1]] = cur_dict # use file_name as the identifier for the dataset
mentioned_data_dicts = dict()
for mentioned_data in mentioned_data_files:
    df = pd.read_csv(mentioned_data, dtype={"userid": str})
    df["userid"] = df["userid"].apply(str.strip)
    cur_dict = df.set_index("userid")["count"].to_dict()
    file_name = followed_data.split("/")
    mentioned_data_dicts[file_name[-1]] = cur_dict


In [8]:
## testing to verify that key and value are right types


# for key, value in most_followed_data_dicts["TMRC15_APAC_3.csv"].items():
#     print(f"Key: {key} ({type(key)}) - Value: {value} ({type(value)})")
# for key, value in retweet_data_dicts["TMRC15_APAC_3.csv"].items():
#     print(f"Key: {key} ({type(key)}) - Value: {value} ({type(value)})")

In [9]:
def process_data(politician_twitter_twitterids):
    politician_twitterid_df = pd.DataFrame(columns=["can_name_clean", "politician_twitterid", "followed_by_dict", "retweet_count_dict", "mentioned_count_dict"])
    for idx, row_data in politician_twitter_twitterids.iterrows():
        twitterid_lst = row_data["twitter"]
        name = row_data["can_name_clean"]
        for twitterid in twitterid_lst.split(";"): # some politicians have multiple twitterids
            # try:
            #     twitterid = int(float(twitterid)) # bypass scientific notation for some of the ids
            # except:
            #     print(f"twitterid: {twitterid} is not a valid Twitter twitterid")
            #     continue
            new_row = pd.DataFrame({ # create new row, where we have a twitterid, a dict for followed_by and retweet_count
                                    # if the twitterid is found in a certain dataset, we will add the dataset as the key and the amt as the value
                'can_name_clean' : name,
                'politician_twitterid': [twitterid],
                "followed_by_dict": [dict()],
                "retweet_count_dict": [dict()],
                "mentioned_count_dict": [dict()]
            })
            for (followed_name, followed_data) in most_followed_data_dicts.items():
                if twitterid in followed_data.keys():
                    new_row.loc[0, "followed_by_dict"][followed_name] = followed_data[twitterid]
            for (retweet_name, retweet_data) in retweet_data_dicts.items():
                if twitterid in retweet_data.keys():
                    new_row.loc[0, "retweet_count_dict"][retweet_name] = retweet_data[twitterid]
            for (mentioned_name, mentioned_data) in mentioned_data_dicts.items():
                if twitterid in mentioned_data.keys():
                    new_row.loc[0, "mentioned_count_dict"][mentioned_name] = mentioned_data[twitterid]           

            politician_twitterid_df = pd.concat([politician_twitterid_df, new_row], ignore_index=True)
    return politician_twitterid_df




In [10]:
politician_twitterid_df_IDN = process_data(politician_twitter_twitterids_IDN)
politician_twitterid_df_MLS = process_data(politician_twitter_twitterids_MLS)
politician_twitterid_df_PHL = process_data(politician_twitter_twitterids_PHL)

In [11]:
def replace_empty_dicts(col):
    return col.apply(lambda x: None if x == {} else x)
## filter out the rows with all empty dicts
politician_twitterid_df_IDN[['followed_by_dict', 'retweet_count_dict', 'mentioned_count_dict']] = politician_twitterid_df_IDN[[
    'followed_by_dict', 'retweet_count_dict', 'mentioned_count_dict']].apply(replace_empty_dicts)
politician_twitterid_df_IDN = politician_twitterid_df_IDN.dropna(subset=["followed_by_dict", "retweet_count_dict", 'mentioned_count_dict'], how="all")
politician_twitterid_df_MLS[['followed_by_dict', 'retweet_count_dict', 'mentioned_count_dict']] = politician_twitterid_df_MLS[[
    'followed_by_dict', 'retweet_count_dict', 'mentioned_count_dict']].apply(replace_empty_dicts)
politician_twitterid_df_MLS = politician_twitterid_df_MLS.dropna(subset=["followed_by_dict", "retweet_count_dict", 'mentioned_count_dict'], how="all")
politician_twitterid_df_PHL[['followed_by_dict', 'retweet_count_dict', 'mentioned_count_dict']] = politician_twitterid_df_PHL[[
    'followed_by_dict', 'retweet_count_dict', 'mentioned_count_dict']].apply(replace_empty_dicts)
politician_twitterid_df_PHL = politician_twitterid_df_PHL.dropna(subset=["followed_by_dict", "retweet_count_dict", 'mentioned_count_dict'], how="all")

In [15]:
## Jaccard's similarity between identified politicians and poltiicians within the region
def jaccards(seta, setb):
    return len(seta.intersection(setb))/len(seta.union(setb))

def politician_percentage (politicians, setb):
    return len(politicians.intersection(setb)) / len(politicians)

politician_twitterids_set_IDN = set(politician_twitter_twitterids_IDN['twitter'])
politician_twitterids_set_MLS = set(politician_twitter_twitterids_MLS['twitter'])
politician_twitterids_set_PHL = set(politician_twitter_twitterids_PHL['twitter'])
politician_twitterids_set_ALL = politician_twitterids_set_PHL.union(politician_twitterids_set_MLS, politician_twitterids_set_IDN)
identified_followed_twitterids_IDN = set(politician_twitterid_df_IDN.set_index("politician_twitterid")["followed_by_dict"].dropna().index)
identified_retweet_twitterids_IDN = set(politician_twitterid_df_IDN.set_index("politician_twitterid")['retweet_count_dict'].dropna().index)
identified_mentioned_twitterids_IDN = set(politician_twitterid_df_IDN.set_index("politician_twitterid")['mentioned_count_dict'].dropna().index)
identified_followed_twitterids_MLS = set(politician_twitterid_df_MLS.set_index("politician_twitterid")['followed_by_dict'].dropna().index)
identified_retweet_twitterids_MLS = set(politician_twitterid_df_MLS.set_index("politician_twitterid")['retweet_count_dict'].dropna().index)
identified_mentioned_twitterids_MLS = set(politician_twitterid_df_MLS.set_index("politician_twitterid")['mentioned_count_dict'].dropna().index)
identified_followed_twitterids_PHL = set(politician_twitterid_df_PHL.set_index("politician_twitterid")['followed_by_dict'].dropna().index)
identified_retweet_twitterids_PHL = set(politician_twitterid_df_PHL.set_index("politician_twitterid")['retweet_count_dict'].dropna().index)
identified_mentioned_twitterids_PHL = set(politician_twitterid_df_PHL.set_index("politician_twitterid")['mentioned_count_dict'].dropna().index)
identified_followed_twitterids_ALL = identified_followed_twitterids_IDN.union(identified_followed_twitterids_MLS, identified_followed_twitterids_PHL)
identified_retweet_twitterids_ALL = identified_retweet_twitterids_IDN.union(identified_retweet_twitterids_MLS, identified_retweet_twitterids_PHL)
identified_mentioned_twitterids_ALL = identified_mentioned_twitterids_IDN.union(identified_mentioned_twitterids_MLS, identified_mentioned_twitterids_PHL)

IDN_followed_jacccard = jaccards(politician_twitterids_set_IDN, identified_followed_twitterids_IDN)
IDN_followed_percentage = politician_percentage(politician_twitterids_set_IDN, identified_followed_twitterids_IDN)
MLS_followed_jacccard = jaccards(politician_twitterids_set_MLS, identified_followed_twitterids_MLS)
MLS_followed_percentage = politician_percentage(politician_twitterids_set_MLS, identified_followed_twitterids_MLS)
PHL_followed_jacccard = jaccards(politician_twitterids_set_PHL, identified_followed_twitterids_PHL)
PHL_followed_percentage = politician_percentage(politician_twitterids_set_PHL, identified_followed_twitterids_PHL)
ALL_followed_jacccard = jaccards(politician_twitterids_set_ALL, identified_followed_twitterids_ALL)
ALL_followed_percentage = politician_percentage(politician_twitterids_set_ALL, identified_followed_twitterids_ALL)


IDN_retweet_jacccard = jaccards(politician_twitterids_set_IDN, identified_retweet_twitterids_IDN)
MLS_retweet_jacccard = jaccards(politician_twitterids_set_MLS, identified_retweet_twitterids_MLS)
PHL_retweet_jacccard = jaccards(politician_twitterids_set_PHL, identified_retweet_twitterids_PHL)
ALL_retweet_jacccard = jaccards(politician_twitterids_set_ALL, identified_retweet_twitterids_ALL)

IDN_mentioned_jacccard = jaccards(politician_twitterids_set_IDN, identified_mentioned_twitterids_IDN)
MLS_mentioned_jacccard = jaccards(politician_twitterids_set_MLS, identified_mentioned_twitterids_MLS)
PHL_mentioned_jacccard = jaccards(politician_twitterids_set_PHL, identified_mentioned_twitterids_PHL)
ALL_mentioned_jacccard = jaccards(politician_twitterids_set_ALL, identified_mentioned_twitterids_ALL)

print(f"IDN_followed_jacccard: {IDN_followed_jacccard}")
print(f"IDN Percentage of Politicians: {IDN_followed_percentage * 100}%")
print(f"MLS_followed_jacccard: {MLS_followed_jacccard}")
print(f"MLS Percentage of Politicians: {MLS_followed_percentage * 100}%")
print(f"PHL_followed_jacccard: {PHL_followed_jacccard}")
print(f"PHL Percentage of Politicians: {PHL_followed_percentage * 100}%")
print(f"ALL_followed_jacccard: {ALL_followed_jacccard}")
print(f"ALL Percentage of Politicians: {ALL_followed_percentage * 100}%")


print(f"IDN_retweet_jacccard: {IDN_retweet_jacccard}")
print(f"MLS_retweet_jacccard: {MLS_retweet_jacccard}")
print(f"PHL_retweet_jacccard: {PHL_retweet_jacccard}")
print(f"ALL_retweet_jacccard: {ALL_retweet_jacccard}")

print(f"IDN_mentioned_jacccard: {IDN_mentioned_jacccard}")
print(f"MLS_mentioned_jacccard: {MLS_mentioned_jacccard}")
print(f"PHL_mentioned_jacccard: {PHL_mentioned_jacccard}")
print(f"ALL_mentioned_jacccard: {ALL_mentioned_jacccard}")

IDN_followed_jacccard: 0.005040322580645161
IDN Percentage of Politicians: 0.5040322580645161%
MLS_followed_jacccard: 0.046370967741935484
MLS Percentage of Politicians: 4.637096774193548%
PHL_followed_jacccard: 0.007281553398058253
PHL Percentage of Politicians: 0.7281553398058253%
ALL_followed_jacccard: 0.01631578947368421
ALL Percentage of Politicians: 1.631578947368421%
IDN_retweet_jacccard: 0.0
MLS_retweet_jacccard: 0.0
PHL_retweet_jacccard: 0.0
ALL_retweet_jacccard: 0.0
IDN_mentioned_jacccard: 0.0
MLS_mentioned_jacccard: 0.0
PHL_mentioned_jacccard: 0.0
ALL_mentioned_jacccard: 0.0


In [13]:
## download data
politician_twitterid_df_IDN.to_csv(f"~/Coding/buntain/politician_twitterid_data/IDN_polticians_identified_twitter.csv", index=False)
politician_twitterid_df_MLS.to_csv(f"~/Coding/buntain/politician_twitterid_data/MLS_polticians_identified_twitter.csv", index=False)
politician_twitterid_df_PHL.to_csv(f"~/Coding/buntain/politician_twitterid_data/PHL_polticians_identified_twitter.csv", index=False)
